##Task 1. Image classification + OOP
In this task, you need to use a publicly available simple MNIST dataset and build 3 classification
models around it. It should be the following models:


1.   Random Forest;
2.   Feed-Forward Neural Network;
3.   Convolutional Neural Network;

Each model should be a separate class that implements $MnistClassifierInterface$ with 2
abstract methods - train and predict. Finally, each of your three models should be hidden under
another $MnistClassifier$ class. $MnistClassifer$ takes an algorithm as an input parameter.
Possible values for the algorithm are: $cnn$, $rf$, and $nn$ for the three models described above.
The solution should contain:
*   Interface for models called $MnistClassifierInterface$.
*   3 classes (1 for each model) that implement $MnistClassifierInterface$.
*   $MnistClassifier$, which takes as an input parameter the name of the algorithm and
provides predictions with exactly the same structure (inputs and outputs) not depending
on the selected algorithm.

####Data

First of all let's import the data. For this I use the Tensorflow library that has a built-in API for the MNIST dataset.

The data in the mnist dataset is represented as matrix of 28 × 28 grayscale pixels that have values from 0 to 255. First of all we need to normalize data, since it enhances training stability and convergence speed but also improves the generalization and accuracy of CNN models.

Then we also need to split data into the training and testing dataset. I use a standard ratio of 4:1 i.e. 80% of data is used for training and 20% for testing.

In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist

SPLIT_PERCENTAGE = 0.8


(images, labels), (_, _) = mnist.load_data()
images = images / 255  # normalising images

print(f"images are of shape: {images.shape} and labels: {labels.shape}")

size = images.shape[0]
split = int(size * SPLIT_PERCENTAGE)
# Subsample the images
train_images = images[:split]
train_labels = labels[:split]

test_images = images[split:]
test_labels = labels[split:]

images are of shape: (60000, 28, 28) and labels: (60000,)


####Classifiers

Now, let's step back and create interface class. For this I use the abcplus library. In this class $MnistClassifierInterface$ I create three abstract methods: $train$ and $predict$ from the conditions of the problem and also $evaluate\_accuracy$ to later compare how different models perform on the same data.

Methods do not contain any implementation. However, I add an implementation of constructor to list all the common protected variables I will be using in the subclasses.

In [2]:
from abc import ABC, abstractmethod

class MnistClassifierInterface(ABC):
    def __init__(self):
        self._train_images = np.ndarray(0)
        self._train_labels = np.ndarray(0)
        self._test_images = np.ndarray(0)
        self._test_labels = np.ndarray(0)
        self._predictions = np.ndarray(0)

    @abstractmethod
    def train(self, train_images: np.ndarray, train_labels: np.ndarray) -> None:
        pass

    @abstractmethod
    def predict(self, test_images: np.ndarray) -> np.ndarray:
        pass

    @abstractmethod
    def evaluate_accuracy(self, test_labels: np.ndarray) -> float:
        pass

For Random Forest classifier I use the sklearn library with its $RandomForestClassifier$ implementation.

However, to work correctly this class needs some additional preprocessing of data. As we originally work with images, they are represented as 2D arrays. But sklearn's classifiers work with 1D arrays only. So we flatten each image into a vector.

For measuring the goodness of performance I use the $accuracy\_score$ from sklearn metrics.

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


class RFClassifier(MnistClassifierInterface):
    def __init__(self):
        self._predictor = RandomForestClassifier()
        super()

    def train(self, train_images: np.ndarray, train_labels: np.ndarray) -> None:
        self._train_images = [image.flatten() for image in train_images]
        self._train_labels = train_labels
        self._predictor.fit(self._train_images, self._train_labels)

    def predict(self, test_images: np.ndarray) -> np.ndarray:
        self._test_images = [image.flatten() for image in test_images]
        self._predictions = self._predictor.predict(self._test_images)
        return self._predictions

    def evaluate_accuracy(self, test_labels: np.ndarray) -> float:
        self._test_labels = test_labels
        return accuracy_score(self._test_labels, self._predictions)


When it comes to neural networks I switch to the Tensorflow library.

For the feed-forward neural network I create a model with only 3 layers. First one takes the input data and flattens it similar to what I manually did in the case with Random Forest. After that there are two fully connected layers, the latter of which has only 10 nodes which corresponds to the number of classes in the dataset (which contains handwritten digits 0-9).

For measurring accuracy score I use the built-in Tensorflow function $evaluate$, that evaluates the performance of the model. This function also does return the loss value, however I disregard it in this task.

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy


class FFNNClassifier(MnistClassifierInterface):
    def __init__(self):
        self.epochs = 5
        self._predictor = Sequential(
            [
                Flatten(input_shape=(28, 28)),
                Dense(128, activation="relu"),
                Dense(10, activation="softmax"),
            ]
        )
        self._predictor.compile(
            optimizer=Adam(),
            loss=SparseCategoricalCrossentropy(),
            metrics=[SparseCategoricalAccuracy()],
        )
        super()

    def train(self, train_images: np.ndarray, train_labels: np.ndarray) -> None:
        self._train_images = train_images
        self._train_labels = train_labels
        self._predictor.fit(
            self._train_images, self._train_labels, epochs=self.epochs, verbose=0
        )

    def predict(self, test_images: np.ndarray) -> np.ndarray:
        self._test_images = test_images
        return self._predictor.predict(self._test_images)

    def evaluate_accuracy(self, test_labels: np.ndarray) -> float:
        self._test_labels = test_labels
        _, test_acc = self._predictor.evaluate(self._test_images, self._test_labels)
        return test_acc



In convolutional neural network classifier I deviate slightly from the initial task and make the $CNNClassifier$ a subclass of $FFNNClassifier$, which in turn implements the $MnistClassifierInterface$, but not a direct implementation of $MnistClassifierInterface$. I do this, because the implementation of all the three methods is identical to feed-forward classifier, and the only thing that is different is the constructor, where we set up the architecture of the model.

For the model itself, I, again, use the same approach as previously, however, I add two pairs of alternating convolutional and pooling layers.

In [5]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

class CNNClassifier(FFNNClassifier):
    def __init__(self):
        self.epochs = 5
        self._predictor = Sequential(
            [
                Conv2D(64, (3, 3), activation="relu", input_shape=(28, 28, 1)),
                MaxPooling2D(2, 2),
                Conv2D(64, (3, 3), activation="relu"),
                MaxPooling2D(2, 2),
                Flatten(),
                Dense(128, activation="relu"),
                Dense(10, activation="softmax"),
            ]
        )
        self._predictor.compile(
            optimizer=Adam(),
            loss=SparseCategoricalCrossentropy(),
            metrics=[SparseCategoricalAccuracy()],
        )
        super()


Now, for the wrapper $MnistClassifier$ class I add a constructor parameter for choosing an algorithm and the $classify$ method that launches the training, prediction and performance evaluation phases of the chosen algorithm.

As a result, I return the predictions that the algorithm made based on the testing data as well as the accuracy score of the model.

In [6]:
from typing import Tuple


class MnistClassifier:
    def __init__(self, algorithm: str):
        match (algorithm):
            case "rf":
                self._classifier = RFClassifier()
            case "nn":
                self._classifier = FFNNClassifier()
            case "cnn":
                self._classifier = CNNClassifier()
            case _:
                raise ValueError("Invalid algorithm")
        self._predictions = np.ndarray(0)
        self._accuracy = 0

    def classify(
        self,
        train_images: np.ndarray,
        train_labels: np.ndarray,
        test_images: np.ndarray,
        test_labels: np.ndarray,
    ) -> Tuple[np.ndarray, float]:
        self._classifier.train(train_images, train_labels)
        self._predictions = self._classifier.predict(test_images)
        self._accuracy = self._classifier.evaluate_accuracy(test_labels)
        return (self._predictions, self._accuracy)



####Results

For the visualization of the work of the three algorithms I create three sepparate objects and launch the respective classifier in each of them. After that I print the accuracy score of each model in color for better contrast.

The predictions array is disregarded as there is no use of it in this case.

In [7]:
class bcolors:
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKCYAN = "\033[96m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

In [8]:
RFClassifier = MnistClassifier("rf")
_, RFAcc = RFClassifier.classify(
    train_images, train_labels, test_images, test_labels
)
print(
    bcolors.OKGREEN + "Random forest accuracy score: " + str(RFAcc) + bcolors.ENDC
)

FFNNClassifier = MnistClassifier("nn")
_, FFNNAcc = FFNNClassifier.classify(
    train_images, train_labels, test_images, test_labels
)
print(
    bcolors.OKGREEN
    + "Fast-forward neural network accuracy score: "
    + str(FFNNAcc)
    + bcolors.ENDC
)

CNNClassifier = MnistClassifier("cnn")
_, CNNAcc = CNNClassifier.classify(
    train_images, train_labels, test_images, test_labels
)
print(
    bcolors.OKGREEN
    + "Convolutional neural network accuracy score: "
    + str(CNNAcc)
    + bcolors.ENDC
)

Random forest accuracy score: 0.9705833333333334


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0936 - sparse_categorical_accuracy: 0.9696
Fast-forward neural network accuracy score: 0.9731666445732117


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0420 - sparse_categorical_accuracy: 0.9880
Convolutional neural network accuracy score: 0.9881666898727417


####Conclusion

In conclusion I can summarize that I successfully implemented the three classifiers that utilize three different classification algorithms: random forest, feed-forward neural network, and convolutional neural network. It is obvious that the complexity of the algorithm grows with each approach. And with the grow of complexity we also witness a slight increase in accuracy of model predictions.